# Composable software from generic code

In the previous notebook we produced the following sum function, which was basically en par in speed with C or numpy:

In [ ]:
function fastsum(v)
    result = zero(eltype(v))
    @simd for i in 1:length(v)    # @simd enforces vectorisation in the loop
        @inbounds result += v[i]  # @inbounds suppresses bound checks
    end
    result
end

But unlike a C implementation, we are not at all restricted to using a particular data type ... and this let's us do crazy things **even though the code is equally fast**:

## Composability

... or the real power of Julia.


(a) **Elevated precision** ... let's consider a nasty case:

In [ ]:
# A vector that sums to one by construction
function generate(N)
    x = randn(N) .* exp.(10 .* randn(N))
    x = [x; -x; 1.0]
    x[sortperm(rand(length(x)))]
end
        
v = generate(10000);

In [ ]:
fastsum(v)

... looks wrong, let's try elevated precision:

In [ ]:
fastsum(big.(v))

(b) **Tracking numerical error**

In [ ]:
import IntervalArithmetic

fastsum(IntervalArithmetic.Interval.(v))

... ups clearly something wrong here! But now we know.

(c) **Error propagation**

In [ ]:
using Measurements
v = [1 ± 1e-10, 1.1 ± 1e-12, 1.2 ± 1e-10]
fastsum(v)

(d) **Unlocking new features**

Let us solve:

$$\frac{du(t)}{dt} = -cu(t)$$

In [ ]:
using OrdinaryDiffEq

# Half-life of Carbon-14 is 5730 years.
c = 5.730

# Setup
u0 = 1.0
tspan = (0.0, 1.0)

# Define the problem
radioactivedecay(u,p,t) = -c*u

# Pass to solver
prob = ODEProblem(radioactivedecay, u0, tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);

In [ ]:
using Plots

plot(sol.t, sol.u, ylabel="u(t)", xlabel="t", lw=2, legend=false)

Now what if we have measurement errors ...

In [ ]:
using OrdinaryDiffEq, Measurements, Plots

# Half-life of Carbon-14 is 5730 years.
c = 5.730 ± 2

#Setup
u0 = 1.0 ± 0.1
tspan = (0.0, 1.0)

#Define the problem
radioactivedecay(u,p,t) = -c*u

#Pass to solver
prob = ODEProblem(radioactivedecay,u0,tspan)
sol = solve(prob, Tsit5(), reltol=1e-8, abstol=1e-8);

plot(sol.t, sol.u, ylabel="u(t)", xlabel="t", lw=2, legend=false)

Note that, in some sense, **Julia implemented that feature by itself**.

The authors of Measurements.jl and DifferentialEquations.jl never had any collabration on this.

It **just works**.

## Generic programming + multiple dispatch

So how does this work?

Julia's ecosystem is centred around convenient abstractions *by convention* (**duck typing**).

- As a **user of a type** we can easily pick within the type hierarchy the most generic type, which satisfies all our needs. If we do not need the precise details of the `Array` implementation, we indicate our **algorithm** works with `AbstractArray`s. The resulting **generic implementation** immediately gives us access to a large range of datatypes.

- As the **provider of a type** we only specify that our object behaves like an `AbstractArray`. Based on providing only a small set of functions in whatever optimised way we want our **new type works with a large range of algorithms**.

In combination with **multiple dispatch** this leads to an ([unreasonable](https://www.youtube.com/watch?v=kc9HwsxE1OY)) amount of code reuse, for example:

1. **Sharing types** (DataStructures.jl, OrderedCollections.jl, StaticArrays.jl, Colors.jl, Measurements.jl)
2. **Sharing algorithms** (StatsBase.jl, SortingAlgorithms.jl, GenericLinearAlgebra.jl)

### Code decoupling by multiple dispatch

Now you might rightfully wonder why I emphasised *multiple dispatch* here. After all this sounds very much just like the generic programming available in other languages. Let me illustrate this with an example.

- We consider the case of a sparse compressed-stored column matrix type `SparseMatrixCSC`, which is available in the `SparseArrays` Julia standard library. Since we care about precision we want to use the `Double64` floating-point type (from `DoubleFloats`), which implements a fast datatype for higher-precision floating-point arithmetic.

- To make these two datastructures interoperate, we need a set of methods. We consider here the scalar-matrix multiplication `*(factor::Double64, matrix::SparseMatrixCSC)`.

- In many languages such as Python, implementing such a binary operator translates to a special operator function. In the Python case this is `__mul__`, which needs to be implemented for the LHS type, i.e. `Double64`. In other words which method to call for `*` is decided solely by considering the type of the *first* argument, hence the name *single dispatch*.

- This is not always the most clever thing to do. In our case one could well argue that `SparseMatrixCSC` should be the place where the operator should be implemented, because the net effect will be to scale this matrix. Amongst other reasons this is why Python rolled out a second set of functions, called `__rmul__`, which dispatch on the *second* argument. Problem solved?

- Well, not really. The issue is that the algorithm in either case ends up being closely coupled to one of the data structures. Moreover since these datastructures live in third-party packages (that know nothing from each other) one either needs to either
   1. Implement `__rmul__` in `SparseMatrixCSC` and add a dependency `SparseArrays` -> `Double64`.
   2. Implement `__mul__` in `Double64`: Same thing in reverse
   3. Derive off e.g. `SparseMatrixCSC` in my code and implement there. Works, but now my code likely ends up in  close coupling to the internals of the `SparseArrays` package.

The aforementioned problem is known as the *binary method problem* and is elegantly solved by multiple dispatch, because both arguments are treated on the same footing. So in Julia I just need to do
```
import Base: *
*(n::Double64, str::SparseMatrixCSC) = ...
```
in my code, without changing any `Double64` or `SparseMatrixCSC` code. Notice how again algorithm and data types are well-separated and can be independently distributed as packages.

- As an asided In C++ the very problem of implementing multiplication operators is not so prominent, because `operator*` can also be a free function. But: For other binary operations (like type conversion) the problem is exactly the same: As soon as two non-standard third-party types are involved and my code needs to connect them, I have to make one more special than the other.

### Workflow advantages by multiple dispatch

Commonly in Julia for each algorithm there exists a generic fallback implementation, which is slow, but always works. An example is the `det`-function we saw earlier. For specific types, where more structure is known, appropriate specialisations are available (like for `StaticArray`s or `Tridiagonal` matrices),
which more efficient.

Let's assume now we program an algorithm, which involves a determinant, e.g. the simple normalisation function
```
normalise(A) = A / det(A)
```
(where of course we assume `A` to be nonsingular). We note:
- As a user, if I know the structure of my matrix is special, I mark it e.g. with the `Tridiagonal` type. When running `normalise` on it, the `det`-method specialised for `Tridiagonal` will be used, so that the best possible performance results.
- As a user, if I do not know the structure of the matrix, I leave it general. Then the slow `det` will still give me a sensible result.
- As a programmer, writing the `normalise` algorithm I do not need to know what kind of matrix my users will pass me, since the dispatch of `det` will assure the best thing happens. Most notably even if I have standard matrices in mind when writing the code, my `normalise` will still work if a passed matrix `A` is non-standard, because the fallback `det` will do the job. 
- As a programmer, writing a custom matrix `A` which adheres to all conventions of the `AbstractMatrix` interface, I instantly have a working `normalise` function even if I could be more clever about it. This way I can rapidly prototype my fancy new matrix type and only later implement the `det` function once I see it's necessary to achieve better performance.
- This allows for a gradual performance increase in codes by **making only local changes** where profiling flags up issues.


**First get it to work, then get it to work *fast*.**

##### More details
- PhD Thesis from Jeff Bezanson
- https://www.youtube.com/watch?v=kc9HwsxE1OY

## Takeaways

- Generic code in julia can be fast
- Everyone can write generic code and everyone should do it.
- Multiple dispatch: Code reuse and emergent features
- Multiple dispatch allows gradual, local performance engineering.